<a href="https://colab.research.google.com/github/kiran-pyt/classification-PyTorch-and-PyTorch-lightning/blob/main/Themodel5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
from torchsummary import summary
from torch.optim.lr_scheduler import StepLR

In [ ]:

train_transforms=transforms.Compose([
                                      transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,), (0.3081,))
])
test_transforms=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
])

In [ ]:
train=datasets.MNIST('./data',train=True,download=True,transform=train_transforms)
test=datasets.MNIST('./data',train=False,download=True,transform=test_transforms)

In [ ]:
SEED=1
cuda=torch.cuda.is_available()
print('cuda',cuda)
torch.manual_seed(SEED)
if cuda:
  torch.cuda.manual_seed(SEED)

dataloader_args=dict(shuffle=True,batch_size=128,num_workers=4,pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)
train_loader=torch.utils.data.DataLoader(train,**dataloader_args)
test_loader=torch.utils.data.DataLoader(test,**dataloader_args)




cuda True


In [ ]:
#train_data=train.train_data
#train_data=train.transform(train_data.numpy())
#print('[Train]')
#print('shape',train_data.cpu().numpy().shape)
#print('size',train.train_data.size())
#print('min',torch.min(train_data))
#print('max',torch.max(train_data))
#print('mean',torch.mean(train_data))
#print('std',torch.std(train_data))
#print('var',torch.var(train_data))

dataiter = iter(train_loader)
images, labels = next(dataiter)

print(images.shape)
print(labels.shape)

# Let's visualize some of the images
%matplotlib inline
import matplotlib.pyplot as plt

plt.imshow(images[0].numpy().squeeze(), cmap='gray_r')

In [ ]:
dropout_value = 0.1
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.convblock1=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=10,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(10),
                             nn.Dropout(dropout_value)

                             ) # output_size = 26
    self.convblock2=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=20,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(20),
                             nn.Dropout(dropout_value)
                             ) # output_size = 24
    self.convblock3=nn.Sequential(nn.Conv2d(in_channels=20,out_channels=10,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(10),
                             nn.Dropout(dropout_value)
                             )  # output_size = 22


    self.pool1=nn.MaxPool2d(2,2) # output_size = 11

    self.convblock4=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=20,kernel_size=(1,1),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(20),
                             nn.Dropout(dropout_value)
                             )   # output_size = 11

    self.convblock5=nn.Sequential(nn.Conv2d(in_channels=20,out_channels=10,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(10),
                             nn.Dropout(dropout_value)
                             )  # output_size = 9


    self.convblock6=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=20,kernel_size=(3,3),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(20),
                             nn.Dropout(dropout_value)
                             ) # output_size = 7
    self.convblock7=nn.Sequential(nn.Conv2d(in_channels=20,out_channels=10,kernel_size=(1,1),padding=0,bias=0),
                             nn.ReLU(),
                             nn.BatchNorm2d(10),
                             nn.Dropout(dropout_value)
                             ) # output_size =7

    self.gap = nn.Sequential(
           nn.AvgPool2d(kernel_size=7) # 7>> 9... nn.AdaptiveAvgPool((1, 1))
        ) # output_size = 1
    self.convblock8=nn.Sequential(nn.Conv2d(in_channels=10,out_channels=10,kernel_size=(1,1),padding=0,bias=0),
                             #nn.ReLU(),
                             #nn.BatchNorm2d(10),
                             #nn.Dropout(dropout_value)
                             ) # output_size =7


  def forward(self,x):
    x=self.convblock1(x)
    x=self.convblock2(x)
    x=self.convblock3(x)
    x=self.pool1(x)
    x=self.convblock4(x)
    x=self.convblock5(x)
    x=self.convblock6(x)
    x=self.convblock7(x)
    x = self.gap(x)
    x=self.convblock8(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=-1)


In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]              90
              ReLU-2           [-1, 10, 26, 26]               0
       BatchNorm2d-3           [-1, 10, 26, 26]              20
           Dropout-4           [-1, 10, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           1,800
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
           Dropout-8           [-1, 20, 24, 24]               0
            Conv2d-9           [-1, 10, 22, 22]           1,800
             ReLU-10           [-1, 10, 22, 22]               0
      BatchNorm2d-11           [-1, 10, 22, 22]              20
          Dropout-12           [-1, 10, 22, 22]               0
        MaxPool2d-13           [-1, 10, 11, 11]               0
           Conv2d-14           [-1

In [ ]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes.
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))

In [ ]:
model =  Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)
scheduler = StepLR(optimizer, step_size=6, gamma=0.1)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

EPOCH: 0


Loss=0.17768621444702148 Batch_id=468 Accuracy=83.95: 100%|██████████| 469/469 [00:23<00:00, 20.32it/s]



Test set: Average loss: 0.0986, Accuracy: 9706/10000 (97.06%)

EPOCH: 1


Loss=0.10188665241003036 Batch_id=468 Accuracy=96.55: 100%|██████████| 469/469 [00:24<00:00, 19.48it/s]



Test set: Average loss: 0.0721, Accuracy: 9802/10000 (98.02%)

EPOCH: 2


...


Test set: Average loss: 0.0274, Accuracy: 9914/10000 (99.14%)

EPOCH: 13


Loss=0.007420410867780447 Batch_id=468 Accuracy=98.69: 100%|██████████| 469/469 [00:22<00:00, 20.90it/s]



Test set: Average loss: 0.0273, Accuracy: 9912/10000 (99.12%)

EPOCH: 14


Loss=0.003657453926280141 Batch_id=468 Accuracy=98.66: 100%|██████████| 469/469 [00:22<00:00, 20.71it/s]



Test set: Average loss: 0.0267, Accuracy: 9918/10000 (99.18%)

